![example](images/director_shot.jpeg)

# PROJECT TITLE: MICROSOFT MOVIE PROJECT 
## Author: Nelson Kibet kemboi 

### OVERVIEW 
In the recent past, there has been increased production of movies in America which has drawn attention to Microsot company. Hence the company has decided to produce thier own movies. Since microsoft company has no idea about movie recording and production, this report will seek to analyse data sets pertaining to movie studio creation and give insights as to why the company should utilize this available venture. In this case we will use exploratory data analysis to pitch recommendations to their stakeholders. 

## Business Problem

In order to invest in this new venture there are factors to be considered. This is important since microsft should not invest in an unworthy project vis a vis their main mandate. This project will find the followig issues hence helping Microsoft map their investment proposal: 
    
***
Questions to consider:
* Finding the best perfoming movies studios. 
* Comparing the cost of production in relation to the revenue earned.
* comparison between various based on ratings and people preference. 
* The microsoft competitors so as to see what best their do
***

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

## DATA CLEANING PROCESS

In [2]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import zipfile
%matplotlib inline

In [3]:
# Here we run the code to unzip and explore the data
with zipfile.ZipFile('zippedData/im.db.zip') as my_zip:
    zipfile.ZipFile.extractall(my_zip, path='ZippedData')
    
# Created a new file called im.db
# Added the file to .gitignore because it's too big to upload to GitHub

### Loading bom.movie_gross.csv

In [7]:
# Creating a connection to the Database
conn = sqlite3.connect('zippedData/im.db')

In [8]:
#connecting to the SQL tables 
pd.read_sql("""
SELECT *
FROM sqlite_schema
WHERE type='table'
""", conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [9]:
# reading first databe 
df1= pd.read_sql("""
SELECT *
FROM movie_ratings;
""", conn)
df1.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [10]:
# readingsecond databe
df2= pd.read_sql("""
SELECT *
FROM movie_basics;
""", conn)
df2.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


### joining the two tables (movie_basics and movie_ratings)

In [12]:
#writing the code to join the two tables 
df3 = pd.read_sql("""
SELECT primary_title, start_year, genres, averagerating, numvotes
FROM movie_basics AS MB
JOIN movie_ratings AS MR
    ON MB.movie_id = mr.movie_id 
    WHERE numvotes> 500000 AND averagerating BETWEEN 6.0 AND 9.5
    ORDER BY averagerating DESC
    LIMIT 50;
""", conn)
df3.head()

,primary_title,start_year,genres,averagerating,numvotes
0,Inception,2010,"Action,Adventure,Sci-Fi",8.8,1841066
1,Interstellar,2014,"Adventure,Drama,Sci-Fi",8.6,1299334
2,The Intouchables,2011,"Biography,Comedy,Drama",8.5,677343
3,Whiplash,2014,"Drama,Music",8.5,616916
4,Avengers: Infinity War,2018,"Action,Adventure,Sci-Fi",8.5,670926


In [13]:
# groupig the data by genre to identify the their frequency based on the views
genres_grouped = df3['genres'].value_counts().head()
genres_grouped

Action,Adventure,Sci-Fi       10
Action,Adventure,Comedy        3
Adventure,Animation,Comedy     3
Adventure,Drama,Fantasy        2
Adventure,Drama,Sci-Fi         2
Name: genres, dtype: int64

In [15]:
#checking for missing values in the limited joined tables 
df3.isna().sum()

primary_title    0
start_year       0
genres           0
averagerating    0
numvotes         0
dtype: int64

It can be seen that the above joined tables have no missing values when the rows are limited by 50

In [16]:
#Identifying the mean of the sorted tables and remove the outlier that is too much spread outside data range 
genres_sorted_mean = pd.DataFrame(df3.groupby("genres")["numvotes"].mean()).sort_values(by="numvotes", ascending= False).iloc[:10]
genres_sorted_mean

,numvotes
genres,
"Action,Thriller",1.387769e+06
"Drama,Western",1.211405e+06
"Biography,Crime,Drama",1.035358e+06
"Mystery,Thriller",1.005960e+06
"Adventure,Drama,Sci-Fi",9.897250e+05
"Action,Adventure,Sci-Fi",8.046928e+05
"Action,Adventure,Comedy",7.712503e+05
"Drama,Mystery,Thriller",7.615920e+05
"Adventure,Family,Fantasy",7.196290e+05


From data combined from two tables i was able to find the most top genres 
when combining the two data sets into one data frame, i filtered the highest rated and most voted movies to come up with a combined data frame (df3).
From this data frame there are no duplicates or null values.
From (df3) dataset I was able to find the top 10 most popular genres with Action,Adventure,Sci-Fi being the leading

### Loading bom.movie_gross.csv

In [19]:
#loading the movie_gross dataset and checking the data overview 
gross_df = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [20]:
#idnetifying missing values on the dataset and converting them to percentages based on null values
gross_df.isna().sum()/len(gross_df)*100

title              0.000000
studio             0.147623
domestic_gross     0.826690
foreign_gross     39.858282
year               0.000000
dtype: float64

It can be seen that movie_gross dataset has some missing values the highest being foreign_gross column with 39% of missing values. The other two are domestic_gross and studio which have lesser missing values.

In [21]:
#checking for duplicates in the data
gross_df.duplicated().sum()

0

From the results above there are no duplicated values in the dataset

In [22]:
#The code below is used to check for null values in the dataset
gross_df[gross_df.isna()]

,title,studio,domestic_gross,foreign_gross,year
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
3382,NaN,NaN,NaN,NaN,NaN
3383,NaN,NaN,NaN,NaN,NaN
3384,NaN,NaN,NaN,NaN,NaN
3385,NaN,NaN,NaN,NaN,NaN


## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [6]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***